In [9]:
import numpy as np

## Generate Data Points
Now we define a function called sample_points for generating our input (x,y) pairs.  
It takes the parameter k as an input which implies number of (x,y) pairs we want to sample. 

In [10]:
def sample_points(k):
    x = np.random.rand(k,50)
    y = np.random.choice([0, 1], size=k, p=[.5, .5]).reshape([-1,1])
    return x,y

In [16]:
x, y = sample_points(10)

print(x[0])
print(y[0])

[0.17523866 0.78666119 0.78959468 0.75941558 0.23837636 0.68746747
 0.69899758 0.61780134 0.23769372 0.22178224 0.0292667  0.15577989
 0.24089443 0.14877691 0.40567656 0.45551465 0.58998332 0.44085659
 0.39054145 0.94724848 0.5970695  0.69266092 0.14627568 0.82849215
 0.88590136 0.63820608 0.7092423  0.27218648 0.88795656 0.61924161
 0.71685824 0.01292618 0.83638255 0.40209115 0.42588467 0.3500624
 0.18305127 0.16302529 0.46201268 0.66012544 0.7527739  0.49131982
 0.91166084 0.79533312 0.65205709 0.76802016 0.66670636 0.80906949
 0.31559429 0.10942784]
[1]


## Single Layer Neural Network
For simplicity and better understand, we use a neural network with only single layer for predicting the output. i.e,

a = np.matmul(X, theta)

YHat = sigmoid(a)

__*So, we use MAML for finding this optimal parameter value theta that is generalizable across tasks. So that 
for a new task, we can learn from a few data points in a lesser time by taking very less gradient steps.*__

## MAML
Now, we define a class called MAML where we implement the MAML algorithm. In the \__init__  method we will initialize all the necessary variables. Then we define our sigmoid activation function. Followed by we define our train function. 

In [21]:
class MAML(object):
    def __init__(self):
        
        #initialize number of tasks i.e number of tasks we need in each batch of tasks
        self.num_tasks = 10
        
        #number of samples i.e number of shots  -number of data points (k) we need to have in each task
        self.num_samples = 10

        #number of epochs i.e training iterations
        self.epochs = 10000
        
        #hyperparameter for the inner loop (inner gradient update)
        self.alpha = 0.0001
        
        #hyperparameter for the outer loop (outer gradient update) i.e meta optimization
        self.beta = 0.0001
       
        #randomly initialize our model parameter theta
        self.theta = np.random.normal(size=50).reshape(50, 1)
      
    #define our sigmoid activation function  
    def sigmoid(self,a):
        return 1.0 / (1 + np.exp(-a))
    
    
    #now let us get to the interesting part i.e training :P
    def train(self):
        
        #for the number of epochs,
        for e in range(self.epochs):        
            
            self.theta_ = []
            
            #for task i in batch of tasks
            for i in range(self.num_tasks):
               
                #sample k data points and prepare our train set
                XTrain, YTrain = sample_points(self.num_samples)
                
                a = np.matmul(XTrain, self.theta)

                YHat = self.sigmoid(a)

                #since we are performing classification, we use cross entropy loss as our loss function
                loss = ((np.matmul(-YTrain.T, np.log(YHat)) - np.matmul((1 -YTrain.T), np.log(1 - YHat)))/self.num_samples)[0][0]
                
                #minimize the loss by calculating gradients
                gradient = np.matmul(XTrain.T, (YHat - YTrain)) / self.num_samples

                #update the gradients and find the optimal parameter theta' for each of tasks
                self.theta_.append(self.theta - self.alpha*gradient)
                
     
            #initialize meta gradients
            meta_gradient = np.zeros(self.theta.shape)
                        
            for i in range(self.num_tasks):
            
                #sample k data points and prepare our test set for meta training
                XTest, YTest = sample_points(10)

                #predict the value of y
                a = np.matmul(XTest, self.theta_[i])
                
                YPred = self.sigmoid(a)
                           
                #compute meta gradients
                meta_gradient += np.matmul(XTest.T, (YPred - YTest)) / self.num_samples

  
            #update our randomly initialized model parameter theta with the meta gradients
            self.theta = self.theta-self.beta*meta_gradient/self.num_tasks
                                       
            if e%1000==0:
                print("Epoch {}: Loss {}\n".format(e,loss))             
                print('Updated Model Parameter Theta\n')
                print('Sampling Next Batch of Tasks \n')
                print('---------------------------------\n')

In [22]:
model = MAML()

In [23]:
model.train()

Epoch 0: Loss 2.0645266148608377

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 1000: Loss 1.2919462201462644

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 2000: Loss 0.8389603336260851

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 3000: Loss 1.8057399065984086

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 4000: Loss 1.9618220084428735

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 5000: Loss 0.8398735491064382

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 6000: Loss 1.3614925202561883

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 7000: Loss 0.8138608045662845

Updated Model Parameter Thet